# Function Tests

This notebook tests `countmatch_common.py`, `countmatch_teps.py` and `countmatch_bagheri.py`.

In [1]:
%matplotlib inline
import sys
sys.path.append('../')
import importlib
import matplotlib.pyplot as plt
import numpy as np
import knowyourdata as kyd

import pandas as pd
from traffic_prophet import cfg
import pathlib, os
import configparser

from traffic_prophet import connection
from traffic_prophet.countmatch import reader
from traffic_prophet.countmatch import growthfactor as gf
from traffic_prophet.countmatch import neighbour

defaultcolours = plt.rcParams['axes.prop_cycle'].by_key()['color']

filepath = pathlib.Path.home().joinpath('.charlesconfig')
if os.path.isfile(filepath):
    vol_conn = connection.Connection(filepath, 'POSTGRES',
                                     'czhu.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'POSTGRES',
                                    'czhu.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['MAPBOX']['token']
    PLOTLY_USER = config['PLOTLY']['user']
    PLOTLY_KEY = config['PLOTLY']['key']
else:
    filepath = pathlib.Path.home().joinpath('cf.txt')
    vol_conn = connection.Connection(filepath, 'localpg',
                                     'prj_vol.btp_centreline_daily_counts')
    ll_conn = connection.Connection(filepath, 'localpg',
                                    'gis.btp_centreline_lonlat')
    config = configparser.RawConfigParser()
    config.read(filepath.as_posix())
    MAPBOX_TOKEN = config['mapbox']['token']
    PLOTLY_USER = config['plotly']['user']
    PLOTLY_KEY = config['plotly']['apikey']

In [2]:
%%time
rdr_teps = reader.Reader(vol_conn)
rdr_teps.read()
gf.get_growth_factors(rdr_teps)

CPU times: user 1min 48s, sys: 2.45 s, total: 1min 50s
Wall time: 2min 1s


In [3]:
ptc_ids = np.unique(np.abs(list(rdr_teps.ptcs.keys())))
nb = neighbour.NeighbourLonLatEuclidean(ll_conn, 20, ptc_ids)
%time nb.find_neighbours()

CPU times: user 18.8 s, sys: 344 ms, total: 19.2 s
Wall time: 18.8 s


In [4]:
%%time
rdr_bag = reader.Reader(vol_conn)
rdr_bag.read()
gf.get_growth_factors(rdr_bag)

CPU times: user 2min 2s, sys: 2.7 s, total: 2min 5s
Wall time: 2min 15s


## TEPs-like CountMatch

Testing the `countmatch_teps` module.

In [7]:
import countmatch_teps as cmt
importlib.reload(cmt)

<module 'countmatch_teps' from '/mnt/c/Users/czhu5/Documents/Ubuntu/GitHub/bdit_traffic_prophet/development_notebooks/countmatch_teps.py'>

In [ ]:
teps_aadt_estimates = cmt.estimate_aadts(
    rdr_teps, nb, 2019, n_neighbours=5, single_direction=True, progress_bar=True)

import countmatch_common as cmc
importlib.reload(cmc)

In [ ]:
ptc_estimates = aadt_estimator_for_ptcs(rdr, 2019, override_growth_factor=False)

In [ ]:
teps_aadt_estimates = cmc.combine_estimators(teps_aadt_estimates, ptc_estimates)
bag_aadt_estimates = cmc.combine_estimators(bag_aadt_estimates, ptc_estimates)